In [1]:
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
# setup disply parameters
import pandas as pd
import numpy as np
from matplotlib import pylab as plt
from matplotlib.ticker import StrMethodFormatter
float_formatter = StrMethodFormatter('{x:0.03f}')
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
SMALL_SIZE = 14
MEDIUM_SIZE = 16
BIGGER_SIZE = 20

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc('figure', figsize=(18, 6)) # set figure size

In [2]:
from abc import abstractmethod, ABC
from dataclasses import dataclass, field
from random import random
from negmas import Mechanism, MechanismRoundResult, Negotiator, AspirationMixin
from negmas import MechanismState
from negmas import AgentMechanismInterface
from negmas import outcome_as_tuple
from typing import Callable, Tuple, Optional, List, Any, Dict
from negmas import Outcome, Issue, UtilityFunction, LinearUtilityFunction, ExpDiscountedUFun
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
import seaborn as sns
import nashpy
from negmas import outcomes as O

In [4]:
class NewsletterUtility(UtilityFunction):
    
    def __init__(self, D, M,  c_time, c_call, D_joint, c_time_joint ):
        super().__init__()
        self.D = D
        self.c_time = c_time
        self.c_call = c_call
        self.D_joint = D_joint
        self.c_time_joint = c_time_joint
        self.M = M
        
        
    def my_S(self,time):
        return (self.D_joint - self.D + self.M*c_call + time*self.c_time - time*self.c_time_joint) -1 
    
    
    def __call__(self, offer):
        #time = offer['time']
        time = offer
        s = self.my_S(time)
        #if self.is_better_tan_Opting_out(time,s):
        return s
        #else: return None
        
    def utility_value(self,time,offer):
        return self.D_joint - offer*self.c_call - time*c_time_joint
    
    def is_better_tan_Opting_out(self,time,s):
        #print('Opt_out',(self.D - self.M*c_call - time*self.c_time),end =" ")
        #print('U',(self.D_joint - s - time*self.c_time_joint),end =" ")
        #print('\n')
        if (self.D - self.M*c_call - time*self.c_time) >= (self.D_joint - s - time*self.c_time_joint):
            #print("OPTING OOUT")
            return False
        return True

       
    def xml(self):
        pass

In [5]:
D1 = 200
D2 = 225

c_time = 1
c_call = 1

D1_joint = 170
D2_joint = 200

c_time_joint = 2

M = 100



A1= NewsletterUtility(D1,M,c_time,c_call,D1_joint,c_time_joint)

A2= NewsletterUtility(D2,M,c_time,c_call,D2_joint,c_time_joint)


#print(F1({'time':10}))
#print(F2({'time':10}))

print(A1(0))
print(A2(0))

print(A1(22))
print(A2(22))

69
74
47
52


In [6]:
from random import sample
import random

class TODNegotiatior(Negotiator):

    def __init__(self, index, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.add_capabilities(dict(propose_for_self=True))
        self.index = index
        self.other = 1 if self.index==0 else 0
        self.Possibles = []
        self.T = 0
        
    def calculate_possible_agreements(self, ufuns: List[UtilityFunction]):
        t=0

        while True:
            offer = ufuns[self.other](t)
            #print(t)
            #print(offer)
            #print(A1.is_better_tan_Opting_out(t,offer2[0]),end="")
            #print(A2.is_better_tan_Opting_out(t,offer2[1]),end="")
            #print('\n')
            if (ufuns[self.index].is_better_tan_Opting_out(t,100-offer) == False) or (ufuns[self.other].is_better_tan_Opting_out(t,offer) == False):
                    break;
            
            s = (100-offer,offer) if self.index == 0 else (offer,100-offer)
            self.Possibles.append(s)
            t +=1  

        self.T = t 

    def propose_for_self(self, ufuns: List[UtilityFunction], time: int) -> List[int]:
        
        offer = ufuns[self.other](time)
        
        o = (100-offer,offer) if self.index == 0 else (offer,100-offer)
        
        if ufuns[self.index].is_better_tan_Opting_out(time,o[self.index]):
            return o
        else: return None
        
        
        

    def accept_offer(self, ufuns: List[UtilityFunction],offer,time):


        if not ufuns[self.index].is_better_tan_Opting_out(offer[self.index],time):
            return False
        
        else:
            for times,offers in enumerate(self.Possibles):
                if ufuns[self.index].utility_value(times,offers[self.index]) > ufuns[self.index].utility_value(time,offer[self.index]):
                    return False

        return True
        
        

In [7]:
from negmas import outcomes as O

class RubinsteinMechanism(Mechanism):
  
    def __init__(self,**kwargs):
        kwargs.update(dict( max_n_agents=2, dynamic_entry=False))
        super().__init__(**kwargs)
        self.add_requirements(dict(propose_for_self = True))
        self.ufuns: List[UtilityFunction] = []

    def add(self,negotiator: "Negotiator",*,ufun: Optional["UtilityFunction"] = None,**kwargs,) -> Optional[bool]:
        added = super().add(negotiator, ufun=ufun, role=None, **kwargs)
        if added:
            self.ufuns.append(self.negotiators[-1].utility_function)
            
    def is_feasible(self,offer):
        if offer[0]<100 and offer[1]<100  and offer[0]>0 and offer[1]>0 :
            return True
        return False

    def round(self)-> MechanismRoundResult:
        """One round of the mechanism"""
        if self.current_step == 0:
            if len(self.negotiators) != 2:
                return MechanismRoundResult(error=True,
                                        error_details=f"Got {len(self.negotiators)} negotiators!!",
                                        broken=True)
            
            [n.calculate_possible_agreements(self.ufuns) for n in self.negotiators]
                
       
        outcomes = list(n.propose_for_self(self.ufuns,self.current_step+1) for n in self.negotiators)
        
        if any(o is None for o in outcomes):
            return MechanismRoundResult(broken=True)
        
        
        if (self.current_step) % 2 == 0: #Turno para Agente1 para hacer oferta
            
            if not self.is_feasible(outcomes[0]):
                return MechanismRoundResult(broken=True)
                
            if  self.negotiators[1].accept_offer(self.ufuns,outcomes[0],self.current_step+1):#Turno para Agente2 para aceptar la oferta
                return MechanismRoundResult(agreement = outcomes[0])
        
        else:#Turno para Agente2 para hacer oferta
            
            if not self.is_feasible(outcomes[1]):
                return MechanismRoundResult(broken=True)
            
            if  self.negotiators[0].accept_offer(self.ufuns,outcomes[1],self.current_step+1):#Turno para Agente1 para aceptar la oferta
                return MechanismRoundResult(agreement = outcomes[1])
            
        return MechanismRoundResult()

In [8]:
D1 = 200
D2 = 225

c_time = 1
c_call = 1

D1_joint = 170
D2_joint = 200

c_time_joint = 2

M = 100


u1= NewsletterUtility(D1,M,c_time,c_call,D1_joint,c_time_joint)

u2= NewsletterUtility(D2,M,c_time,c_call,D2_joint,c_time_joint)


m = RubinsteinMechanism()

m.add(TODNegotiatior(0 ,ufun=u1, name="a1"))
m.add(TODNegotiatior(1, ufun=u2, name="a2" ))



result = m.run()
print(f"Agreed to: {result.agreement} in {m.current_step} steps")

Agreed to: None in 22 steps
